In [1]:
# Sentiment Analysis
'''
    Model : bert-base
    Dataset : Rotten Tomato Movie Review Dataset
    Info : 0 -> Negative / 1 -> Positive
'''

'\n    Model : bert-base\n    Dataset : Rotten Tomato Movie Review Dataset\n    Info : 0 -> Negative / 1 -> Positive\n'

##### Configuration

In [2]:
!pip install transformers

In [3]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)

# Get the data

In [ ]:
# URL = "https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"

# dataset = tf.keras.utils.get_file(fname="aclImdb_v1.tar.gz", 
#                                   origin=URL,
#                                   untar=True,
#                                   cache_dir='.',
#                                   cache_subdir='')

In [ ]:
# # The shutil module offers a number of high-level operations on files and collections of files.
# import os
# import shutil
# # Create main directory path ("/aclImdb")
# main_dir = os.path.join(os.path.dirname(dataset), 'aclImdb')
# # Create sub directory path ("/aclImdb/train")
# train_dir = os.path.join(main_dir, 'train')
# # Remove unsup folder since this is a supervised learning task
# remove_dir = os.path.join(train_dir, 'unsup')
# shutil.rmtree(remove_dir)
# # View the final train folder
# print(os.listdir(train_dir))

In [4]:
# We create a training dataset and a validation 
# dataset from our "aclImdb/train" directory with a 80/20 split.
train = tf.keras.preprocessing.text_dataset_from_directory(
    'aclImdb/train', batch_size=30000, validation_split=0.2, 
    subset='training', seed=123)
test = tf.keras.preprocessing.text_dataset_from_directory(
    'aclImdb/train', batch_size=30000, validation_split=0.2, 
    subset='validation', seed=123)

Found 55482 files belonging to 3 classes.
Using 44386 files for training.
Found 55482 files belonging to 3 classes.
Using 11096 files for validation.


In [5]:
import pandas as pd
for i in train.take(1):
  train_feat = i[0].numpy()
  train_lab = i[1].numpy()

train = pd.DataFrame([train_feat, train_lab]).T
train.columns = ['text', 'label']
train['text'] = train['text'].str.decode("utf-8")
train.head()

,text,label
0,I have seen many good Korean Movies including ...,0
1,"Okay... it seems like so far, only the Barman ...",0
2,If Edward Woodward was the the flicks watching...,0
3,"What a film, excellent is not the word for it....",2
4,I went into this expecting not to like it; I f...,1


In [6]:
for j in test.take(1):
  test_feat = j[0].numpy()
  test_lab = j[1].numpy()

test = pd.DataFrame([test_feat, test_lab]).T
test.columns = ['text', 'label']
test['text'] = test['text'].str.decode("utf-8")
test.head()

,text,label
0,For anyone with even a fleeting interest in hu...,2
1,Awesome blend of love and suspense. A top clas...,2
2,I wasn't expecting THEY LIVE BY NIGHT. I wasn'...,2
3,This movie in away was super-clever. It's them...,0
4,StarCraft is a real-time strategy game (RTS) s...,2


# Loading pre-trained BERT

In [7]:
from transformers import BertTokenizer, TFBertForSequenceClassification

In [8]:
# Loading Tkenizer and Model (bert-base-uncased)
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = TFBertForSequenceClassification.from_pretrained("bert-base-uncased")

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
tokenizer

BertTokenizer(name_or_path='bert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True)

In [10]:
model

In [11]:
model.summary()

Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  109482240 
                                                                 
 dropout_37 (Dropout)        multiple                  0         
                                                                 
 classifier (Dense)          multiple                  1538      
                                                                 
Total params: 109,483,778
Trainable params: 109,483,778
Non-trainable params: 0
_________________________________________________________________


# Prepare the data

#### Steps for preparing data : 
#### 1) Filtering / removing un-necessary words/tokens from dataset
#### 2) Tokenization
#### 3) Padding
#### 4) Positional Embeddings / Word Embeddings

In [ ]:
# from sklearn.model_selection import train_test_split
# train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

In [ ]:
# print(train_df.shape, test_df.shape)

In [ ]:
# train_df["text"][0]

In [ ]:
# train_df["label"][0]

In [12]:
from transformers import InputExample, InputFeatures

def convert_data_to_examples(train, test, data, label):
    train_input_examples = train.apply(lambda x: InputExample(
        guid = None,
        text_a = x[data],
        text_b = None,
        label = x[label]
    ), axis=1)
    test_input_examples = test.apply(lambda x: InputExample(
        guid = None,
        text_a = x[data],
        text_b = None,
        label = x[label]
    ), axis=1)
    
    
    return train_input_examples, test_input_examples


train_input_examples, test_input_examples = convert_data_to_examples(train, test, 'text', 'label')

In [21]:
train_input_examples

0        InputExample(guid=None, text_a="I have seen ma...
1        InputExample(guid=None, text_a='Okay... it see...
2        InputExample(guid=None, text_a="If Edward Wood...
3        InputExample(guid=None, text_a='What a film, e...
4        InputExample(guid=None, text_a="I went into th...
                               ...                        
29995    InputExample(guid=None, text_a="Today's kids, ...
29996    InputExample(guid=None, text_a="This is the fo...
29997    InputExample(guid=None, text_a='Not since Ed W...
29998    InputExample(guid=None, text_a='Like "Six Feet...
29999    InputExample(guid=None, text_a='I love watchin...
Length: 30000, dtype: object

In [22]:
test_input_examples

0        InputExample(guid=None, text_a='For anyone wit...
1        InputExample(guid=None, text_a='Awesome blend ...
2        InputExample(guid=None, text_a='I wasn\'t expe...
3        InputExample(guid=None, text_a='This movie in ...
4        InputExample(guid=None, text_a='StarCraft is a...
                               ...                        
11091    InputExample(guid=None, text_a='Woody Harrelso...
11092    InputExample(guid=None, text_a='This cowardly ...
11093    InputExample(guid=None, text_a="`How To Lose A...
11094    InputExample(guid=None, text_a="I had been try...
11095    InputExample(guid=None, text_a="Hey Arnold! is...
Length: 11096, dtype: object

In [ ]:
# def convert_examples_to_tf_dataset(examples, tokenizer, max_length=128):
#     features = []
    
#     for example in examples:
#         input_dict = tokenizer.encode_plus(
#             example.text_a,
#             add_special_tokens = True,
#             max_length = max_length,
#             return_token_type_ids = True,
#             return_attention_mask = True,
#             pad_to_max_length = True,
#             truncation = True
#         )
# #         method 'input_dict' returns a dictionary with following keys
#         input_ids, token_type_ids, attention_mask = (input_dict["input_ids"], input_dict["token_type_ids"], input_dict["attention_mask"])
    
#         features.append(
#             InputFeatures(
#                 input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids, label=example.label
#             )
#         )
        
#         def gen():
#             for f in features:
#                 yield(
#                     {
#                         "input_ids":f.input_ids,
#                         "token_type_ids":f.token_type_ids,
#                         "attention_mask":f.attention_mask,
#                     },
#                     f.label,
#                 )
                
#         return tf.data.Dataset.from_generator(
#             gen,
#             ({"input_ids": tf.int32, "attention_mask": tf.int32, "token_type_ids": tf.int32}, tf.int64),
#             (
#                 {
#                     "input_ids": tf.TensorShape([None]),
#                     "attention_mask": tf.TensorShape([None]),
#                     "token_type_ids": tf.TensorShape([None]),
#                 },
#                 tf.TensorShape([]),
#             ),
#         ) 

In [ ]:
sentence = "My name is rajesh."
print(tokenizer.tokenize(sentence))

In [29]:
import tensorflow as tf
def convert_examples_to_tf_dataset(examples, tokenizer, max_length=128):
    features = []
    
    for example in examples:
        input_dict = tokenizer.encode_plus(
            list(example.text_a),
            add_special_tokens = True,
            max_length = max_length,
            return_token_type_ids = True,
            return_attention_mask = True,
            pad_to_max_length = True,
            truncation = True
        )
#         method 'input_dict' returns a dictionary with following keys
        input_ids, token_type_ids, attention_mask = (input_dict["input_ids"], input_dict["token_type_ids"], input_dict["attention_mask"])
    
        features.append(
            InputFeatures(
                input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids, label=example.label
            )
        )
        
        def gen():
            for f in features:
                yield(
                    {
                        "input_ids":f.input_ids,
                        "token_type_ids":f.token_type_ids,
                        "attention_mask":f.attention_mask,
                    },
                    f.label,
                )
                
    return tf.data.Dataset.from_generator(
            gen,
            ({"input_ids": tf.int32, "attention_mask": tf.int32, "token_type_ids": tf.int32}, tf.int64),
            (
                {
                    "input_ids": tf.TensorShape([None]),
                    "attention_mask": tf.TensorShape([None]),
                    "token_type_ids": tf.TensorShape([None]),
                },
                tf.TensorShape([]),
            ),
        )

In [30]:
train_data = convert_examples_to_tf_dataset(list(train_input_examples),tokenizer)
test_data = convert_examples_to_tf_dataset(list(test_input_examples),tokenizer)
train_data = train_data.shuffle(100).batch(32).repeat(2)
test_data = test_data.batch(32)

In [31]:
train_data

<RepeatDataset element_spec=({'input_ids': TensorSpec(shape=(None, None), dtype=tf.int32, name=None), 'attention_mask': TensorSpec(shape=(None, None), dtype=tf.int32, name=None), 'token_type_ids': TensorSpec(shape=(None, None), dtype=tf.int32, name=None)}, TensorSpec(shape=(None,), dtype=tf.int64, name=None))>

In [32]:
test_data

<BatchDataset element_spec=({'input_ids': TensorSpec(shape=(None, None), dtype=tf.int32, name=None), 'attention_mask': TensorSpec(shape=(None, None), dtype=tf.int32, name=None), 'token_type_ids': TensorSpec(shape=(None, None), dtype=tf.int32, name=None)}, TensorSpec(shape=(None,), dtype=tf.int64, name=None))>

In [33]:
for data, label in train_data:
    print('Input IDs:', data['input_ids'])
    print('Attention Mask:', data['attention_mask'])
    print('Token Type IDs:', data['token_type_ids'])
    print('Labels:', label)

Input IDs: tf.Tensor(
[[ 101  100 1037 ... 1051  100  102]
 [ 101  100 1048 ... 1049  100  102]
 [ 101  100 1037 ... 1055 1037  102]
 ...
 [ 101  100 1044 ... 1048 1048  102]
 [ 101  100  100 ... 1051 1039  102]
 [ 101  100 1042 ... 1054 1061  102]], shape=(32, 128), dtype=int32)
Attention Mask: tf.Tensor(
[[1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]
 ...
 [1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]], shape=(32, 128), dtype=int32)
Token Type IDs: tf.Tensor(
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]], shape=(32, 128), dtype=int32)
Labels: tf.Tensor([0 0 2 1 0 2 0 1 2 2 1 2 2 0 0 2 2 2 0 1 1 2 1 2 2 2 2 2 2 2 0 0], shape=(32,), dtype=int64)
Input IDs: tf.Tensor(
[[ 101  100  100 ... 1041 1040  102]
 [ 101  100 1044 ...  100  100  102]
 [ 101  100  100 ... 1041 1039  102]
 ...
 [ 101  100 1044 ... 1061 1051  102]
 [ 101  100 1041 ...  100 1054  102]
 [ 101  100  100 ...  100 1056  102]], s

Input IDs: tf.Tensor(
[[ 101  100 1005 ...  100  100  102]
 [ 101  100 1042 ... 1039 1039  102]
 [ 101  100 1044 ...  100  100  102]
 ...
 [ 101 1015 1014 ... 1054 1054  102]
 [ 101  100 1005 ... 1061  100  102]
 [ 101  100 1044 ... 1045 1056  102]], shape=(32, 128), dtype=int32)
Attention Mask: tf.Tensor(
[[1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]
 ...
 [1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]], shape=(32, 128), dtype=int32)
Token Type IDs: tf.Tensor(
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]], shape=(32, 128), dtype=int32)
Labels: tf.Tensor([2 1 1 1 1 0 2 2 2 1 1 2 2 2 2 2 1 2 2 2 1 0 2 2 1 0 2 1 2 2 2 1], shape=(32,), dtype=int64)
Input IDs: tf.Tensor(
[[ 101  100 1056 ... 1051 1042  102]
 [ 101  100  100 ... 1041 1054  102]
 [ 101  100 1051 ... 1045 1041  102]
 ...
 [ 101  100  100 ... 1037 1054  102]
 [ 101  100 1050 ... 1037 1056  102]
 [ 101  100 1045 ... 1045 1056  102]], s

Input IDs: tf.Tensor(
[[ 101  100  100 ... 1045 1050  102]
 [ 101  100 1044 ... 1045 1050  102]
 [ 101  100  100 ... 1054 1045  102]
 ...
 [ 101  100 1045 ... 1055 1055  102]
 [ 101  100 1037 ... 1061  100  102]
 [ 101  100 1044 ... 1041  100  102]], shape=(32, 128), dtype=int32)
Attention Mask: tf.Tensor(
[[1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]
 ...
 [1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]], shape=(32, 128), dtype=int32)
Token Type IDs: tf.Tensor(
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]], shape=(32, 128), dtype=int32)
Labels: tf.Tensor([2 0 2 2 2 0 2 2 1 2 1 1 2 2 2 0 0 0 1 1 2 2 1 2 2 2 1 2 2 1 0 2], shape=(32,), dtype=int64)
Input IDs: tf.Tensor(
[[ 101  100  100 ...  100 1056  102]
 [ 101  100 1044 ... 1058 1041  102]
 [ 101  100 1044 ... 1040 1041  102]
 ...
 [ 101  100 1054 ... 1045 1040  102]
 [ 101  100 1037 ... 1056 1044  102]
 [ 101  100 1005 ... 1039 1054  102]], s

Input IDs: tf.Tensor(
[[ 101  100 1042 ... 1037 1061  102]
 [ 101  100 1044 ... 1044 1045  102]
 [ 101  100 1044 ... 1037 1050  102]
 ...
 [ 101  100 1044 ... 1041 1054  102]
 [ 101  100 1044 ... 1038 1054  102]
 [ 101  100  100 ...  100 1046  102]], shape=(32, 128), dtype=int32)
Attention Mask: tf.Tensor(
[[1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]
 ...
 [1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]], shape=(32, 128), dtype=int32)
Token Type IDs: tf.Tensor(
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]], shape=(32, 128), dtype=int32)
Labels: tf.Tensor([2 2 0 1 0 1 2 0 0 2 2 0 2 2 0 2 2 1 1 0 0 1 2 2 2 0 1 2 2 1 1 0], shape=(32,), dtype=int64)
Input IDs: tf.Tensor(
[[ 101  100  100 ... 1054  100  102]
 [ 101  100 1044 ...  100 1055  102]
 [ 101  100 1044 ... 1050 1061  102]
 ...
 [ 101 1059 1041 ...  100 1049  102]
 [ 101  100 1051 ... 1037 1038  102]
 [ 101  100 1051 ... 1057 1038  102]], s

Input IDs: tf.Tensor(
[[ 101  100  100 ...  100 1038  102]
 [ 101  100 1037 ...  100 1045  102]
 [ 101 1005  100 ... 1051 1050  102]
 ...
 [ 101 1005  100 ... 1055  100  102]
 [ 101  100 1044 ... 1018 1010  102]
 [ 101  100 1044 ... 1056 1044  102]], shape=(32, 128), dtype=int32)
Attention Mask: tf.Tensor(
[[1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]
 ...
 [1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]], shape=(32, 128), dtype=int32)
Token Type IDs: tf.Tensor(
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]], shape=(32, 128), dtype=int32)
Labels: tf.Tensor([2 0 2 2 2 2 1 1 2 2 2 0 1 2 2 2 2 2 1 0 2 2 2 0 2 2 2 2 2 2 2 2], shape=(32,), dtype=int64)
Input IDs: tf.Tensor(
[[ 101  100 1005 ... 1044 1045  102]
 [ 101  100 1044 ...  100 1000  102]
 [ 101 1045 1055 ...  100 1038  102]
 ...
 [ 101  100 1051 ... 1056  100  102]
 [ 101  100  100 ... 1047 1041  102]
 [ 101  100 1058 ... 1042 1042  102]], s

...

Input IDs: tf.Tensor(
[[ 101  100  100 ...  100 1013  102]
 [ 101  100 1041 ... 1048 1040  102]
 [ 101  100 1045 ... 1056 1051  102]
 ...
 [ 101  100 1050 ...  100 1050  102]
 [ 101  100  100 ... 1056 1045  102]
 [ 101  100 1044 ...  100 1045  102]], shape=(32, 128), dtype=int32)
Attention Mask: tf.Tensor(
[[1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]
 ...
 [1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]], shape=(32, 128), dtype=int32)
Token Type IDs: tf.Tensor(
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]], shape=(32, 128), dtype=int32)
Labels: tf.Tensor([2 2 2 1 0 2 2 1 0 0 2 1 0 1 2 2 2 1 2 2 2 1 2 2 1 0 2 1 2 2 2 2], shape=(32,), dtype=int64)
Input IDs: tf.Tensor(
[[ 101  100  100 ... 1039  100  102]
 [ 101  100 1051 ... 1041 1050  102]
 [ 101  100  100 ... 1040  100  102]
 ...
 [ 101  100 1044 ... 1056 1044  102]
 [ 101  100 1041 ...  100 1054  102]
 [ 101  100 1044 ... 1040 1051  102]], s

Input IDs: tf.Tensor(
[[ 101  100 1044 ...  100 1051  102]
 [ 101  100 1037 ... 1039 1044  102]
 [ 101 1000  100 ... 1056 1044  102]
 ...
 [ 101 1050 1041 ... 1041  100  102]
 [ 101  100 1037 ... 1049 1041  102]
 [ 101  100 1048 ...  100 1045  102]], shape=(32, 128), dtype=int32)
Attention Mask: tf.Tensor(
[[1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]
 ...
 [1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]], shape=(32, 128), dtype=int32)
Token Type IDs: tf.Tensor(
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]], shape=(32, 128), dtype=int32)
Labels: tf.Tensor([1 1 0 0 2 2 2 1 2 1 1 0 2 0 0 2 0 2 2 2 1 0 2 2 2 1 0 1 0 2 1 0], shape=(32,), dtype=int64)
Input IDs: tf.Tensor(
[[ 101  100 1037 ... 1050 1056  102]
 [ 101  100  100 ... 1037 1039  102]
 [ 101  100 1054 ... 1056 1057  102]
 ...
 [ 101  100  100 ... 1041  100  102]
 [ 101  100 1051 ... 1037  100  102]
 [ 101  100 1057 ... 1040 1037  102]], s

Input IDs: tf.Tensor(
[[ 101  100 1044 ... 1056 1039  102]
 [ 101 1026 1038 ... 1057 1039  102]
 [ 101  100  100 ... 1039 1012  102]
 ...
 [ 101  100 1056 ... 1049 1051  102]
 [ 101  100  100 ... 1056 1010  102]
 [ 101  100  100 ... 1057 1055  102]], shape=(32, 128), dtype=int32)
Attention Mask: tf.Tensor(
[[1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]
 ...
 [1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]], shape=(32, 128), dtype=int32)
Token Type IDs: tf.Tensor(
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]], shape=(32, 128), dtype=int32)
Labels: tf.Tensor([2 2 2 2 1 0 2 1 1 2 2 1 2 2 2 1 2 1 2 2 1 0 1 0 0 2 0 1 2 2 0 2], shape=(32,), dtype=int64)
Input IDs: tf.Tensor(
[[ 101  100 1005 ...  100 1037  102]
 [ 101  100  100 ... 1041 1045  102]
 [ 101  100  100 ... 1062 1041  102]
 ...
 [ 101  100 1051 ... 1052 1051  102]
 [ 101  100 1051 ... 1042  100  102]
 [ 101  100  100 ...  100  100  102]], s

Input IDs: tf.Tensor(
[[ 101  100 1051 ...  100 1059  102]
 [ 101  100 1044 ... 1055 1056  102]
 [ 101  100 1041 ...  100 1055  102]
 ...
 [ 101  100 1044 ... 1054  100  102]
 [ 101  100 1044 ...  100 1039  102]
 [ 101  100 1042 ... 1054 1045  102]], shape=(32, 128), dtype=int32)
Attention Mask: tf.Tensor(
[[1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]
 ...
 [1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]], shape=(32, 128), dtype=int32)
Token Type IDs: tf.Tensor(
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]], shape=(32, 128), dtype=int32)
Labels: tf.Tensor([0 2 0 2 0 2 2 1 2 0 2 2 0 2 2 1 1 2 1 2 0 2 1 1 0 1 2 0 2 1 0 2], shape=(32,), dtype=int64)
Input IDs: tf.Tensor(
[[ 101  100 1042 ...  100 1051  102]
 [ 101 1000  100 ... 1039 1044  102]
 [ 101  100 1044 ... 1058 1041  102]
 ...
 [ 101  100  100 ...  100 1059  102]
 [ 101 1000  100 ... 1039 1041  102]
 [ 101  100 1037 ... 1042 1051  102]], s

Input IDs: tf.Tensor(
[[ 101  100 1044 ... 1055  100  102]
 [ 101  100 1044 ...  100 1056  102]
 [ 101  100 1044 ... 1050  100  102]
 ...
 [ 101  100 1051 ... 1041 1054  102]
 [ 101  100 1044 ... 1039 1056  102]
 [ 101  100 1037 ... 1037  100  102]], shape=(32, 128), dtype=int32)
Attention Mask: tf.Tensor(
[[1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]
 ...
 [1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]], shape=(32, 128), dtype=int32)
Token Type IDs: tf.Tensor(
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]], shape=(32, 128), dtype=int32)
Labels: tf.Tensor([2 1 1 0 0 2 2 1 0 2 2 0 0 2 1 2 0 2 2 2 2 0 2 1 0 2 1 2 0 0 2 0], shape=(32,), dtype=int64)
Input IDs: tf.Tensor(
[[ 101  100  100 ... 1041  100  102]
 [ 101  100  100 ... 1054 1044  102]
 [ 101  100  100 ...  100 1049  102]
 ...
 [ 101  100 1044 ... 1010  100  102]
 [ 101  100 1056 ... 1000  100  102]
 [ 101  100 1044 ... 1000  100  102]], s

In [34]:
count = 0
for _ in train_data:
    count += 1

print("Dataset Size:", count)

Dataset Size: 1876


In [35]:
count = 0
for _ in test_data:
    count += 1

print("Dataset Size:", count)

Dataset Size: 347


In [ ]:
# Our data is now ready to be fed into the model

# Select a model to train

In [36]:
# Configuration of our model
from keras.optimizers import Adam
from keras.losses import SparseCategoricalCrossentropy
from keras.metrics import SparseCategoricalAccuracy
model.compile(optimizer=Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0), 
              loss=SparseCategoricalCrossentropy(from_logits=True), 
              metrics=[SparseCategoricalAccuracy('accuracy')])

In [17]:
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [37]:
model.fit(train_data, epochs=2, validation_data=test_data)

Epoch 1/2


ResourceExhaustedError: Graph execution error:

Detected at node 'tf_bert_for_sequence_classification/bert/encoder/layer_._1/intermediate/Gelu/mul_1' defined at (most recent call last):
    File "C:\Users\manas\anaconda3\envs\tf\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\Users\manas\anaconda3\envs\tf\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "C:\Users\manas\anaconda3\envs\tf\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "C:\Users\manas\anaconda3\envs\tf\lib\site-packages\traitlets\config\application.py", line 992, in launch_instance
      app.start()
    File "C:\Users\manas\anaconda3\envs\tf\lib\site-packages\ipykernel\kernelapp.py", line 711, in start
      self.io_loop.start()
    File "C:\Users\manas\anaconda3\envs\tf\lib\site-packages\tornado\platform\asyncio.py", line 215, in start
      self.asyncio_loop.run_forever()
    

# Fine tune the model

# Test (test.csv)

In [ ]:
pred_sentences = ['I grew up (b. 1965) watching and loving the Thunderbirds. All my mates at school watched. We played Thunderbirds before school, during lunch and after school. We all wanted to be Virgil or Scott. No one wanted to be Alan. Counting down from 5 became an art form. I took my children to see the movie hoping they would get a glimpse of what I loved as a child. How bitterly disappointing. The only high point was the snappy theme tune. Not that it could compare with the original score of the Thunderbirds. Thankfully early Saturday mornings one television channel still plays reruns of the series Gerry Anderson and his wife created. Jonatha Frakes should hand in his directors chair, his version was completely hopeless. A waste of film. Utter rubbish. A CGI remake may be acceptable but replacing marionettes with Homo sapiens subsp. sapiens was a huge error of judgment.']

In [ ]:
tf_batch = tokenizer(pred_sentences, max_length=128, padding=True, truncation=True, return_tensors='tf')
tf_outputs = model(tf_batch)
tf_predictions = tf.nn.softmax(tf_outputs[0], axis=-1)
labels = ['Negative','Positive']
label = tf.argmax(tf_predictions, axis=1)
label = label.numpy()
for i in range(len(pred_sentences)):
  print(pred_sentences[i], ": \n", labels[label[i]])

# Deployment (AWS/Gradio)